In [17]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

reg = LinearRegression(fit_intercept=False)

In [ ]:
'''Part A'''

def simulation1(N, gamma, beta):

    sample_mean = np.zeros(3)

    sample_covariance = np.array([
        [1, 0, 0],
        [0, 1, 0.5],
        [0, 0.5, 1]
    ])

    samples = np.random.multivariate_normal(sample_mean, sample_covariance, N)
    X, W, U = samples[:, 0], samples[:, 1], samples[:, 2]
    
    Z = np.where(X > 0, 1, 0)
    D = np.where(gamma[0] + gamma[1]*Z + W > 0, 1, 0)
    Y = beta[0] + beta[1]*D + U
    
    results = {
        'Y': Y,
        'D': D,
        'W': W,
        'X': X,
        'Z': Z,
        'U': U,
    }

    return results

In [10]:
'''Part B'''

simulate_b = simulation1(10000, [0.25, 0.25], [0.5, 1])
correlation_b = np.corrcoef(simulate_b['D'], simulate_b['U'])[0, 1]
print(f"Correlation between D and U: {correlation_b:.4f}")

Correlation between D and U: 0.3867


In [23]:
'''Part C'''
def intercept(X):
    return np.column_stack([np.ones(len(X)), X])

reg.fit(intercept(simulate_b['D']), simulate_b['Y'])
beta_0 = reg.coef_[0]
beta_1 = reg.coef_[1]

print(f"β₀ (intercept): {beta_0:.4f}")
print(f"β₁ (treatment effect): {beta_1:.4f}")
print(f"Interpretation: On average, being treated (D=1) increases outcome Y by {beta_1:.4f} units")
print(f"compared to not being treated (D=0). The baseline outcome when D=0 is {beta_0:.4f}.")

β₀ (intercept): -0.0252
β₁ (treatment effect): 1.8031
Interpretation: On average, being treated (D=1) increases outcome Y by 1.8031 units
compared to not being treated (D=0). The baseline outcome when D=0 is -0.0252.


In [27]:
'''Part D'''
reg.fit(intercept(simulate_b['Z']), simulate_b['D'])
alpha_0 = reg.coef_[0]
alpha_1 = reg.coef_[1]

print(f"α₀ (intercept): {alpha_0:.4f}")
print(f"α₁ (instrument effect): {alpha_1:.4f}")
print(f"Interpretation:")
print(f"- α₀: {alpha_0:.1%} of people with Z=0 receive treatment (D=1)")
print(f"- α₁: Having Z=1 increases probability of treatment by {alpha_1:.1%}")
print(f"- Total: {alpha_0 + alpha_1:.1%} of people with Z=1 receive treatment")

α₀ (intercept): 0.5941
α₁ (instrument effect): 0.0942
Interpretation:
- α₀: 59.4% of people with Z=0 receive treatment (D=1)
- α₁: Having Z=1 increases probability of treatment by 9.4%
- Total: 68.8% of people with Z=1 receive treatment


In [30]:
'''Part E'''
reg.fit(intercept(simulate_b['X']), simulate_b['D'])
delta_0 = reg.coef_[0]
delta_1 = reg.coef_[1]

print(f"δ₀ (intercept): {delta_0:.4f}")
print(f"δ₁ (effect of X on D): {delta_1:.4f}")
print(f"Interpretation:")
print(f"- δ₀: {delta_0:.1%} of people with X=0 receive treatment (D=1)")
print(f"- δ₁: A one-unit increase in X increases probability of treatment by {delta_1:.1%}")
print(f"- Total: {delta_0 + delta_1:.1%} of people with X=1 receive treatment")

δ₀ (intercept): 0.6412
δ₁ (effect of X on D): 0.0352
Interpretation:
- δ₀: 64.1% of people with X=0 receive treatment (D=1)
- δ₁: A one-unit increase in X increases probability of treatment by 3.5%
- Total: 67.6% of people with X=1 receive treatment


Comparatively, Z is shown to have a more significant positive effect on D.

In [ ]:
'''Part F'''
def IV_estimate(X, Y, Z):

    covXZ = np.cov(X, Z, bias=True)[0, 1]
    covYZ = np.cov(Y, Z, bias=True)[0, 1]
    # First stage: Regress X on Z
    reg.fit(intercept(Z), X)
    X_hat = reg.predict(intercept(Z))
    
    # Second stage: Regress Y on predicted X
    reg.fit(intercept(X_hat), Y)
    return reg.coef_[1]

iv_beta_z = IV_estimate(simulate_b['D'], simulate_b['Y'], simulate_b['Z'])
print(f"IV estimate of β₁ using Z as instrument: {iv_beta_z:.4f}")

iv_beta_x = IV_estimate(simulate_b['D'], simulate_b['Y'], simulate_b['X'])
print(f"IV estimate of β₁ using X as instrument: {iv_beta_x:.4f}")

IV estimate of β₁ using Z as instrument: 1.1579
IV estimate of β₁ using X as instrument: 1.3369
